In [1]:
import requests
from requests.auth import HTTPBasicAuth
import json
import base64
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [2]:
auth = "Bearer OTc3MTkyMjk5NzE4OhR/SuR7NZfuGfgkMKEKAgiRLHZ9"

In [3]:
# Replace BaseURL and project key with your own project
board_url = 'https://issues.apache.org/jira/rest/agile/1.0/board?projectKeyOrId=USERGRID'

headers = {
   "Accept": "application/json",
   "Authorization": auth,
}

payload = {
   "maxResults":50 #100, 20
}

# Request data by using GET method and directly get the json output
board = requests.request("GET",board_url,headers=headers, params=payload).json()

# As we only intreseted in the "values" in the board, so we only normalize the board['values']
board_df = pd.json_normalize(board["values"])

# Filter the scrum board type and maintain the pd dataframe formate
board_df = board_df[board_df['type']=='scrum']

# Reset the index for the future convience.
board_df = board_df.reset_index()

# Store the board id into variable board_id
board_id = board_df['id']

#Save the df as csv file. Use your project name to define the file name
board_df.to_csv('USERGRID_board.csv') 

In [4]:
headers = {
   "Content-Type": "application/json",
   "Authorization": auth,  
}


payload_sprint = {
'maxResults':2000  #Retrieve at most 2000 records, can be modified based on your own project
}

list1 = [] 

# Loop through the required board, and store the records into the empty list1
for i in board_id:
   sprint_url = 'https://issues.apache.org/jira/rest/agile/1.0/board/' + f'{i}' + '/sprint'
   sprint = requests.request("GET",sprint_url ,headers=headers, params=payload_sprint).json()
   try:
      list1.append(sprint['values'])
   except:
      pass

In [5]:
# Normalize the list and get the inital dataframe, whose record are nested.
sprint_df = pd.json_normalize(list1)
sprint_df.head()

,0,1,2,3,4,5,6,7,8,9,...,28,29,30,31,32,33,34,35,36,37
0,"{'id': 1, 'self': 'https://issues.apache.org/j...","{'id': 60, 'self': 'https://issues.apache.org/...","{'id': 61, 'self': 'https://issues.apache.org/...","{'id': 64, 'self': 'https://issues.apache.org/...","{'id': 70, 'self': 'https://issues.apache.org/...","{'id': 71, 'self': 'https://issues.apache.org/...","{'id': 78, 'self': 'https://issues.apache.org/...","{'id': 79, 'self': 'https://issues.apache.org/...","{'id': 82, 'self': 'https://issues.apache.org/...","{'id': 83, 'self': 'https://issues.apache.org/...",...,"{'id': 176, 'self': 'https://issues.apache.org...","{'id': 177, 'self': 'https://issues.apache.org...","{'id': 178, 'self': 'https://issues.apache.org...","{'id': 186, 'self': 'https://issues.apache.org...","{'id': 210, 'self': 'https://issues.apache.org...","{'id': 187, 'self': 'https://issues.apache.org...","{'id': 218, 'self': 'https://issues.apache.org...","{'id': 221, 'self': 'https://issues.apache.org...","{'id': 224, 'self': 'https://issues.apache.org...","{'id': 226, 'self': 'https://issues.apache.org..."


In [6]:
#Create an empty dataframe
sprint_result = pd.DataFrame()
sprint_result.head()

# sprint_df.index is the row number. For each row, we need to flatten the records and store them into a dataframe. Our goal is to flatten all these record and generate a single dataframe.
for i in sprint_df.index:
   # Flatten records in a row based on row index
   sprint_df_1 = sprint_df.iloc[i].apply(pd.Series) 
   # Attach corresponding board_id
   sprint_df_1['board_id'] = board_df['id'][i]
   # Create a list called "frame" to tell python we want to contact two dataframs together -- sprint_result and sprint_df_1
   frame = [sprint_result, sprint_df_1]
   #Contact two dataframes
   sprint_result = pd.concat(frame)

In [7]:
# Filter the sprint that totally empty
sprint_result = sprint_result[sprint_result['id'].notna()]
sprint_result.head()

,id,self,state,name,startDate,endDate,completeDate,activatedDate,originBoardId,board_id
0,1,https://issues.apache.org/jira/rest/agile/1.0/...,closed,Usergrid 0,2013-07-10T01:38:13.706Z,2013-07-24T01:38:13.706Z,2015-01-03T20:42:56.150Z,2013-07-10T01:38:13.706Z,NaN,23
1,60,https://issues.apache.org/jira/rest/agile/1.0/...,closed,Usergrid 1,2015-01-05T17:30:41.106Z,2015-01-19T17:30:00.000Z,2015-01-12T17:03:43.474Z,2015-01-05T17:30:41.106Z,23.0,23
2,61,https://issues.apache.org/jira/rest/agile/1.0/...,closed,Usergrid 2,2015-01-12T17:17:07.193Z,2015-01-26T17:17:00.000Z,2015-01-20T17:46:08.454Z,2015-01-12T17:17:07.193Z,23.0,23
3,64,https://issues.apache.org/jira/rest/agile/1.0/...,closed,Usergrid 3,2015-01-20T17:51:37.934Z,2015-02-03T17:51:00.000Z,2015-01-26T18:15:04.654Z,2015-01-20T17:51:37.934Z,23.0,23
4,70,https://issues.apache.org/jira/rest/agile/1.0/...,closed,Usergrid 4,2015-01-26T18:15:37.031Z,2015-02-09T18:15:00.000Z,2015-02-02T20:03:29.485Z,2015-01-26T18:15:37.031Z,23.0,23


In [8]:
# Reset the index
sprint_result = sprint_result.reset_index()
# Drop the origianl index column
sprint_result = sprint_result.drop(columns=['index'])
sprint_result.head()

,id,self,state,name,startDate,endDate,completeDate,activatedDate,originBoardId,board_id
0,1,https://issues.apache.org/jira/rest/agile/1.0/...,closed,Usergrid 0,2013-07-10T01:38:13.706Z,2013-07-24T01:38:13.706Z,2015-01-03T20:42:56.150Z,2013-07-10T01:38:13.706Z,NaN,23
1,60,https://issues.apache.org/jira/rest/agile/1.0/...,closed,Usergrid 1,2015-01-05T17:30:41.106Z,2015-01-19T17:30:00.000Z,2015-01-12T17:03:43.474Z,2015-01-05T17:30:41.106Z,23.0,23
2,61,https://issues.apache.org/jira/rest/agile/1.0/...,closed,Usergrid 2,2015-01-12T17:17:07.193Z,2015-01-26T17:17:00.000Z,2015-01-20T17:46:08.454Z,2015-01-12T17:17:07.193Z,23.0,23
3,64,https://issues.apache.org/jira/rest/agile/1.0/...,closed,Usergrid 3,2015-01-20T17:51:37.934Z,2015-02-03T17:51:00.000Z,2015-01-26T18:15:04.654Z,2015-01-20T17:51:37.934Z,23.0,23
4,70,https://issues.apache.org/jira/rest/agile/1.0/...,closed,Usergrid 4,2015-01-26T18:15:37.031Z,2015-02-09T18:15:00.000Z,2015-02-02T20:03:29.485Z,2015-01-26T18:15:37.031Z,23.0,23


In [9]:
# Save as csv file
sprint_result.to_csv('USERGRID_sprint.csv')

In [10]:
headers = {
   "Content-Type": "application/json",
   "Authorization": auth,  
}

payload_issues = {
'maxResults':3000
}

list1 = []
# Loop through the required board, and store the records into the empty list1
for i in board_id:
   issues_url = 'https://issues.apache.org/jira/rest/agile/1.0/board/' + f'{i}' + '/issue'
   issues = requests.request("GET",issues_url ,headers=headers, params=payload_issues).json()
   try:
      list1.append(issues['issues'])
   except:
      pass

In [11]:
# Normalize the list and get the inital dataframe, whose record are nested.
issue_df = pd.json_normalize(list1)
issue_df.head()

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,"{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...",...,"{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation..."


In [12]:
issue_result = pd.DataFrame()

for i in issue_df.index:
   issue_df_1 = issue_df.iloc[i].apply(pd.Series)
   issue_df_1['board_id'] = board_df['id'][i]
   frame = [issue_result, issue_df_1]
   issue_result = pd.concat(frame)

issue_result = issue_result.reset_index()
issue_result = issue_result.drop(columns=['index'])

In [13]:
#Only select issues that have story points
issue_result = issue_result[issue_result['fields.customfield_12310293'].notna()]
issue_result.head()

,expand,id,self,key,fields.fixVersions,fields.resolution.self,fields.resolution.id,fields.resolution.description,fields.resolution.name,fields.customfield_12312322,...,fields.parent.fields.issuetype.self,fields.parent.fields.issuetype.id,fields.parent.fields.issuetype.description,fields.parent.fields.issuetype.iconUrl,fields.parent.fields.issuetype.name,fields.parent.fields.issuetype.subtask,fields.parent.fields.issuetype.avatarId,fields.timetracking.timeSpent,fields.timetracking.timeSpentSeconds,board_id
4,"operations,versionedRepresentations,editmeta,c...",12913744,https://issues.apache.org/jira/rest/agile/1.0/...,USERGRID-1118,[{'self': 'https://issues.apache.org/jira/rest...,https://issues.apache.org/jira/rest/api/2/reso...,1,A fix for this issue is checked into the tree ...,Fixed,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23
5,"operations,versionedRepresentations,editmeta,c...",12937571,https://issues.apache.org/jira/rest/agile/1.0/...,USERGRID-1256,[{'self': 'https://issues.apache.org/jira/rest...,https://issues.apache.org/jira/rest/api/2/reso...,1,A fix for this issue is checked into the tree ...,Fixed,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23
6,"operations,versionedRepresentations,editmeta,c...",12910492,https://issues.apache.org/jira/rest/agile/1.0/...,USERGRID-1093,[],https://issues.apache.org/jira/rest/api/2/reso...,2,The problem described is an issue which will n...,Won't Fix,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23
9,"operations,versionedRepresentations,editmeta,c...",12844533,https://issues.apache.org/jira/rest/agile/1.0/...,USERGRID-832,[{'self': 'https://issues.apache.org/jira/rest...,https://issues.apache.org/jira/rest/api/2/reso...,10,As the name suggests,Implemented,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23
11,"operations,versionedRepresentations,editmeta,c...",12941082,https://issues.apache.org/jira/rest/agile/1.0/...,USERGRID-1268,[],https://issues.apache.org/jira/rest/api/2/reso...,11,,Done,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23


In [14]:
#Check the dataframe shape
issue_result.shape

(428, 253)

In [15]:
#Save as csv file
issue_result.to_csv('USERGRID_issues.csv')